In [17]:
from google.colab import files
uploaded = files.upload()


Saving heart_disease_uci.csv to heart_disease_uci (1).csv


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
#Loading the uploaded dataset
df = pd.read_csv("heart_disease_uci.csv") 
df.head()  # Preview the first few rows


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [21]:
#Data Cleaning and Preprocessing

# Convert 'num' column to binary labels: 0 = no disease, 1 = disease present
df['num'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

# Drop columns with too many missing values to simplify preprocessing
df.drop(columns=['ca', 'thal', 'slope'], inplace=True)

# Fill missing values in numeric columns with the median (robust to outliers)
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(['id'])
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Encode categorical columns (like 'sex', 'cp', 'restecg') into numeric values
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoders in case you need to decode later

# Drop irrelevant columns like ID and dataset name which don't help prediction
df.drop(columns=['id', 'dataset'], inplace=True)


In [22]:
#Split the data into features (X) and target (y)
X = df.drop('num', axis=1)  # All columns except the target
y = df['num']  # Target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
#Define and compare multiple classification models

# Initialize models in a dictionary
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictions on the test set

    # Print evaluation results
    print(f"\n{name} Results:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))



Logistic Regression Results:
Accuracy: 0.8043478260869565
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.84      0.78        75
           1       0.88      0.78      0.83       109

    accuracy                           0.80       184
   macro avg       0.80      0.81      0.80       184
weighted avg       0.81      0.80      0.81       184

Confusion Matrix:
 [[63 12]
 [24 85]]

K-Nearest Neighbors Results:
Accuracy: 0.6956521739130435
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.67      0.64        75
           1       0.76      0.72      0.74       109

    accuracy                           0.70       184
   macro avg       0.69      0.69      0.69       184
weighted avg       0.70      0.70      0.70       184

Confusion Matrix:
 [[50 25]
 [31 78]]

Random Forest Results:
Accuracy: 0.8206521739130435
Classification Report:
               precision    

Results:
Random Forest achieved the highest accuracy (~82%) and balanced precision-recall.
- It effectively identified patients at risk of heart disease with fewer false negatives compared to other models.